In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.applications import EfficientNetB0  



In [2]:

TRAIN_DIR = "train"        
TEST_DIR = "test"          
LABELS_CSV = "labels.csv"  
SAMPLE_SUB = "sample_submission.csv"

In [3]:
labels = pd.read_csv(LABELS_CSV)
sample_sub = pd.read_csv(SAMPLE_SUB)


In [4]:
labels

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


In [5]:
labels["filename"] = labels["id"].astype(str) + ".jpg"
labels.head()

,id,breed,filename
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,00214f311d5d2247d5dfe4fe24b2303d.jpg
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,0021f9ceb3235effd7fcde7f7538ed62.jpg


In [6]:
train_df,val_df=train_test_split(labels,test_size=0.2,random_state=42)

In [7]:
train_df

,id,breed,filename
6887,ad019e394b55406f412b9a03899d5705,welsh_springer_spaniel,ad019e394b55406f412b9a03899d5705.jpg
7975,c77266278126119fe02a3550dd0c3c1c,norfolk_terrier,c77266278126119fe02a3550dd0c3c1c.jpg
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher,ffe5f6d8e2bff356e9482a80a6e29aac.jpg
4695,75dbdbf6814e6c72a7cab75c73a5acd7,leonberg,75dbdbf6814e6c72a7cab75c73a5acd7.jpg
940,16fdb4a145a7f5695a264ba980e23bc3,norwich_terrier,16fdb4a145a7f5695a264ba980e23bc3.jpg
...,...,...,...
5734,90ce42cb4fa91a9e5709c799021e0e38,beagle,90ce42cb4fa91a9e5709c799021e0e38.jpg
5191,83217877553b0d7ba501cf416e4b7b8e,standard_poodle,83217877553b0d7ba501cf416e4b7b8e.jpg
5390,8810923d4134c41b7ca0e8543c31ffeb,italian_greyhound,8810923d4134c41b7ca0e8543c31ffeb.jpg
860,14f25ad78f02126a1ceeb44385378cd1,great_pyrenees,14f25ad78f02126a1ceeb44385378cd1.jpg


In [8]:
print("Train:", len(train_df), "Val:", len(val_df))


Train: 8177 Val: 2045


In [9]:
IMG_SIZE = (128, 128)  
BATCH_SIZE = 32        


In [10]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=TRAIN_DIR,
    x_col="filename",
    y_col="breed",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 8177 validated image filenames belonging to 120 classes.


In [13]:
val_datagenenrator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=TRAIN_DIR,
    x_col="filename",
    y_col="breed",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 2045 validated image filenames belonging to 120 classes.


CHECKING SECTION

In [14]:
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)

Number of classes: 120


In [17]:
print(f"Class indices: {train_generator.class_indices}")

Class indices: {'affenpinscher': 0, 'afghan_hound': 1, 'african_hunting_dog': 2, 'airedale': 3, 'american_staffordshire_terrier': 4, 'appenzeller': 5, 'australian_terrier': 6, 'basenji': 7, 'basset': 8, 'beagle': 9, 'bedlington_terrier': 10, 'bernese_mountain_dog': 11, 'black-and-tan_coonhound': 12, 'blenheim_spaniel': 13, 'bloodhound': 14, 'bluetick': 15, 'border_collie': 16, 'border_terrier': 17, 'borzoi': 18, 'boston_bull': 19, 'bouvier_des_flandres': 20, 'boxer': 21, 'brabancon_griffon': 22, 'briard': 23, 'brittany_spaniel': 24, 'bull_mastiff': 25, 'cairn': 26, 'cardigan': 27, 'chesapeake_bay_retriever': 28, 'chihuahua': 29, 'chow': 30, 'clumber': 31, 'cocker_spaniel': 32, 'collie': 33, 'curly-coated_retriever': 34, 'dandie_dinmont': 35, 'dhole': 36, 'dingo': 37, 'doberman': 38, 'english_foxhound': 39, 'english_setter': 40, 'english_springer': 41, 'entlebucher': 42, 'eskimo_dog': 43, 'flat-coated_retriever': 44, 'french_bulldog': 45, 'german_shepherd': 46, 'german_short-haired_poin

In [18]:
with open("class_indices.txt", "w") as f:
    for class_name, index in train_generator.class_indices.items():
        f.write(f"{class_name}: {index}\n")
print(f"Class indices saved to class_indices.txt")

Class indices saved to class_indices.txt


In [ ]:
def build_model(input_shape=(128,128,3),n_clssess=num_classes):
    model=model.Sequential()

  
    model.add(layers.conv2D(32,(3,3),activation='relu',padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(0.25))

  
    model.add(layers.conv2D(64,(3,3),activation='relu',padding='same')) 
    model.add(layers.BatchNormalization())
    model.add(layers.conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(layers.MaxPooling2D((2,2)))   
    model.add(layers.Dropout(0.25))

    
    model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))  

    model.add(layers.Dense(num_classes, activation='softmax'))
    return model
